<a href="https://colab.research.google.com/github/arupnayak/llama2-faiss-chat/blob/master/whisper_large_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Installation

In [2]:
!pip install --q --upgrade pip
!pip install --q --upgrade git+https://github.com/huggingface/transformers.git accelerate datasets[audio]
!pip install --q flash-attn --no-build-isolation

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 8.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 38.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ibis-framework 7.1.0 requires pyarrow<15,>=2, but you have pyarrow 15.0.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 22.0 MB/s eta 

### Import required packages

In [3]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline

In [4]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

In [5]:
device

'cuda:0'

In [6]:
torch_dtype

torch.float16

In [7]:
model_id = "openai/whisper-large-v3"

In [8]:
model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
)
model.to(device)

processor = AutoProcessor.from_pretrained(model_id)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.90k [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/340 [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.07k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [9]:
pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    max_new_tokens=128,
    chunk_length_s=15,
    batch_size=16,
    return_timestamps=True,
    torch_dtype=torch_dtype,
    device=device,
)

In [10]:
pipe

In [23]:
audio = "HIN_M_DeepakS.mp3"

In [24]:
%%timeit
result = pipe(audio) # v2 gives better results if you don't provide a language.

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentiall

4.88 s ± 326 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [21]:
result = pipe(audio)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


In [22]:
result['text']

' To many, traditional ways of farming represent an ideal way of life, an antidote to this modern technological age. Yet everywhere they are under threat, not from the Western world, nor from technology, but from simple mathematics.'

In [18]:
result['chunks']

[{'timestamp': (1.0, 2.0), 'text': ' New job?'},
 {'timestamp': (2.0, 4.0), 'text': ' So new life?'},
 {'timestamp': (4.0, 6.0), 'text': ' Then why stay the old you?'},
 {'timestamp': (6.0, 8.0), 'text': ' Go new! Go colour!'},
 {'timestamp': (8.0, 10.0), 'text': ' With new Garnier Colour Naturals'},
 {'timestamp': (10.0, 12.0), 'text': ' Discover a new you'},
 {'timestamp': (12.0, 14.0), 'text': ' Rich, vibrant, stunning colours,'},
 {'timestamp': (14.0, 16.0), 'text': ' enriched with three precious oils'},
 {'timestamp': (16.0, 18.0), 'text': ' for up to twice more care.'},
 {'timestamp': (18.0, 20.0), 'text': ' Your hair loves the nourishment'},
 {'timestamp': (20.0, 22.0), 'text': ' and we love the colour.'},
 {'timestamp': (22.0, 27.98),
  'text': ' Ready to try the new you? Garnier Color Naturals. Go New. Go Color.'}]

In [ ]:
%%timeit
result = pipe(audio, generate_kwargs={"language": "english"})

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentiall

8.65 s ± 647 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
# translation

# result = pipe(audio,
#               return_timestamps=True,
#               generate_kwargs={"language": "french", "task": "translate"})


# print(result["chunks"])

In [ ]:
result = pipe(audio, return_timestamps=True)
print(result["chunks"])


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


[{'timestamp': (0.0, 5.12), 'text': ' In a previous video, I showed you a number of demos where people were using GPT-4 with'}, {'timestamp': (5.12, 9.6), 'text': ' vision and the OpenAI text-to-speech API to create video narrators.'}, {'timestamp': (10.08, 15.28), 'text': " So I'll walk you through a step-by-step process of how exactly to do that inside a google collab"}, {'timestamp': (15.28, 20.16), 'text': " notebook everything you're about to see is based on this cookbook from openair i'm going to put"}, {'timestamp': (20.16, 23.18), 'text': ' a link to this in the description. Okay, so here is'}, {'timestamp': (23.22, 26.16), 'text': ' how the workflow is going to look like. We will provide an'}, {'timestamp': (26.16, 31.22), 'text': ' input video. Now GPT-4 does not understand videos by itself,'}, {'timestamp': (31.74, 36.94), 'text': ' we will have to convert those into frames and then feed those frames into GPT-4'}, {'timestamp': (36.94, 45.4), 'text': ' with vision. Then in G

In [ ]:
# result = pipe(audio, return_timestamps="word")
# print(result["chunks"])


### Use Flash Attention

In [ ]:
# model = AutoModelForSpeechSeq2Seq.from_pretrained(
#     model_id,
#     torch_dtype=torch_dtype,
#     low_cpu_mem_usage=True,
#     use_safetensors=True,
#     use_flash_attention_2=True
#     )

# model.to(device)


In [ ]:
# !pip install --upgrade optimum


In [ ]:
# pipe = pipeline(
#     "automatic-speech-recognition",
#     model=model,
#     tokenizer=processor.tokenizer,
#     feature_extractor=processor.feature_extractor,
#     max_new_tokens=128,
#     chunk_length_s=15,
#     batch_size=16,
#     return_timestamps=True,
#     torch_dtype=torch_dtype,
#     device=device,
# )

#Distill Whisper

In [ ]:
# !pip install -q transformers accelerate datasets[audio]
# #!pip install -q flash-attn --no-build-isolation
# !pip install --q optimum

In [ ]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
# from optimum.bettertransformer import BetterTransformer

In [ ]:
model_id = "distil-whisper/distil-medium.en"

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True#, use_flash_attention_2=True
)
model.to(device)


(…)istil-medium.en/resolve/main/config.json:   0%|          | 0.00/2.26k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/789M [00:00<?, ?B/s]

(…)m.en/resolve/main/generation_config.json:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

WhisperForConditionalGeneration(
  (model): WhisperModel(
    (encoder): WhisperEncoder(
      (conv1): Conv1d(80, 1024, kernel_size=(3,), stride=(1,), padding=(1,))
      (conv2): Conv1d(1024, 1024, kernel_size=(3,), stride=(2,), padding=(1,))
      (embed_positions): Embedding(1500, 1024)
      (layers): ModuleList(
        (0-23): 24 x WhisperEncoderLayer(
          (self_attn): WhisperAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=False)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=Tru

In [ ]:
processor = AutoProcessor.from_pretrained(model_id)

pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    max_new_tokens=128,
    chunk_length_s=15,
    batch_size=16,
    torch_dtype=torch_dtype,
    device=device,
)

(…)en/resolve/main/preprocessor_config.json:   0%|          | 0.00/339 [00:00<?, ?B/s]

(…)um.en/resolve/main/tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

(…)distil-medium.en/resolve/main/vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

(…)il-medium.en/resolve/main/tokenizer.json:   0%|          | 0.00/2.41M [00:00<?, ?B/s]

(…)distil-medium.en/resolve/main/merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

(…)l-medium.en/resolve/main/normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

(…)medium.en/resolve/main/added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

(…).en/resolve/main/special_tokens_map.json:   0%|          | 0.00/2.06k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
%%timeit
result = pipe(audio)

6.19 s ± 581 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
result = pipe(audio)

In [ ]:
result

{'text': " In a previous video, I showed you a number of demos where people were using JPD 4 with vision and the open AI text to speech API to create video narrators. So I'll walk you through a step by step process of how exactly to do that inside a Google CoLab notebook. Everything you're about to see is based on this bookbook from Open Air. I'm going to put a link to this in the description. Okay, so here is how the workflow is going to look like. We will provide an input video. Now, GPD4 does not understand videos by itself. We will have to convert those into frames and then feed those frames into GPT4 with vision. Then in GPT4 with vision will generate a description, all the frames, so whatever it sees in the frames. Then we will use the newly released text to speech API from Open AI to convert that description into speech. And then we will need to combine both the original input video, as well as the generated audio, to create a new video. Now, I'll walk you through a step-by by s